Automação Web e Busca de Informação com Python

Desafio:

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação do:
- Dolar
- Euro
- Ouro

Precisamos pegar na internet, de forma automatica, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Para isso vamos criar uma automação Web.
    - Usaremos o selenium
    - Importante: baixar o Webdriver
         - Chrome chromedriver
         - Baixar e colocar dentro do arquivo do Anaconda

Web Scraping: Pesquisar informações na web

In [47]:
from selenium import webdriver
# keys é o comando que usa o teclado em segundo plano
from selenium.webdriver.common.keys import Keys

# Selenium permite integrar o Python com o computador

# abrir o navegador automaticamente
navegador = webdriver.Chrome()
# Com mais algumas linhas de códigos, é possível não ver o navegador
# Deixar unicamente em segundo plano


# Para o navegador funcionar em segundo plano:

# from selenium.webdriver.chrome.options import Options
# chrome_options = Options
# chrome_options.headless = True  # also works
# nav = webdriver.Chrome(options = chrome_options)


# Passo a passo:
# OBS: No sites eles não usam \ (contra barra) usa / (barra)
# não precisa usar o r


In [48]:
# pesquisar no google cotação dolar
# pegar a cotação do dolar
navegador.get('https://www.google.com/')

# abrir o navegador -> Inspesionar -> 
# Setinha no canto superior esquerdo do inspesionar
# Clica na barra de pesquisa -> Ver onde ficou selecionado
# Botão esquerdo -> Copy -> Copy XPath -> Colar código abaixo

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegando o valor da busca do google
# get.attribute() para pegar o valor do link
# tem que procurar no link onde está esse valor
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

5.438336


In [49]:
# pesquisar no google cotação euro
# pegar a cotação do euro

navegador.get('https://www.google.com/')

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)


6.53685268


In [50]:
# entrar no site melhor combio
# pegar a cotação do ouro

navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
print(cotacao_ouro)

# tratar o texto: trocando a , (virgula) pelo . (ponto)
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

309,14
309.14


In [51]:
# Para clicar em algum elemento da página

# Mesmo passo a passo para achar o Copy XPath
# navegador.find_element_by_xpath('endereço copy XPath').click()

pyautogui -> automatiza mouse, tela e teclado
- São automações que impede você de usar o computador
- Funciona em qualquer ferramenta/sistema/programa
- Se a tela mudar, ele para de funcionar

selenium -> automatiza a web/internet
- Só funciona para internet
- Funciona mesmo com você fazendo outra coisa no computador
- Funcina mesmo se a posição das coisas na tela mudar

Tentar usar o selenium sempre que possível

Agora vamos atualiza a nossa base de preços com as novas cotações
- Importando a base de dados

In [52]:
import pandas as pd

# Arquivo excel na pasta do arquivo Aula 4
tabela_produtos = pd.read_excel('Produtos.xlsx')
display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


Tabela
    Preço base Original é o preço que eu compro
    Preço base Reais é o preço em reais 
    Ajuste é o ajuste para eu ter o lucro
    Preço Final, é o preço de venda

- Atualizando os preços e o cálculo do Preço Final

In [53]:
# Passo 1: Atualizar as cotações

# Por usar [] (Colchetes) por padrão ele usa as colunas
# Não preciso usar axis=1
# loc é para localizar 
# loc[Na coluna de moeda onde está escrito dolar, 'coluna'] que eu pretendo editar
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)

# Atualizando o Euro
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)

# Atualizando o Ouro
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)


# Passo 2: Atualizar o preço base reais -> cotação * preço base original
tabela_produtos['Preço Base Reais'] = tabela_produtos['Cotação'] * tabela_produtos['Preço Base Original']
# tabela_produtos['Preço Base Reais'] = tabela_produtos['Preço Base Reais'].map('{:.2f}'.format)

# Passo 3:
tabela_produtos['Preço Final'] = tabela_produtos['Ajuste'] * tabela_produtos['Preço Base Reais']

# Passo 4: Formatando as casas decimais
tabela_produtos['Preço Final'] = tabela_produtos['Preço Final'].map('{:.2f}'.format)
tabela_produtos['Preço Base Reais'] = tabela_produtos['Preço Base Reais'].map('{:.2f}'.format)
tabela_produtos['Cotação'] = tabela_produtos['Cotação'].map('{:.4f}'.format)

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.4383,5438.28,1.40,7613.59
1,Carro Renault,4500.00,Euro,6.5369,29415.84,2.00,58831.67
2,Notebook Dell,899.99,Dólar,5.4383,4894.45,1.70,8320.56
3,IPhone,799.00,Dólar,5.4383,4345.23,1.70,7386.89
4,Carro Fiat,3000.00,Euro,6.5369,19610.56,1.90,37260.06
5,Celular Xiaomi,480.48,Dólar,5.4383,2613.01,2.00,5226.02
6,Joia 20g,20.00,Ouro,309.1400,6182.80,1.15,7110.22


Agora vamos exportar a nova base de preços atualizada

In [54]:
# Para exportar essa base de dados 
# Tomar cuidado para não deixar o mesmo nome do outra tabela 
# Caso contrário, irá substituir automaticamente
# Para não importar o index junto com a tabela, index=False
tabela_produtos.to_excel('Produtos Atualisado.xlsx', index=False)